<a href="https://colab.research.google.com/github/Nada-Nasser/images-classification/blob/main/image_classification_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install colabcode

In [ ]:
pip install torchvision


In [ ]:
pip install nltk

In [ ]:
!wget https://raw.githubusercontent.com/Lasagne/Recipes/master/examples/resnet50/imagenet_classes.txt

--2021-08-09 19:42:37--  https://raw.githubusercontent.com/Lasagne/Recipes/master/examples/resnet50/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21674 (21K) [text/plain]
Saving to: ‘imagenet_classes.txt.2’

imagenet_classes.tx 100%[===================>]  21.17K  --.-KB/s    in 0.001s  

2021-08-09 19:42:37 (17.0 MB/s) - ‘imagenet_classes.txt.2’ saved [21674/21674]



In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

In [ ]:
pip install fastapi

In [ ]:
from colabcode import ColabCode
from fastapi import FastAPI
import uvicorn

In [ ]:
cc=ColabCode(port=12000,code=False)

In [ ]:
import base64
from pydantic import BaseModel
from PIL import Image

class Data(BaseModel):
    file: str

app=FastAPI()

@app.post("/classifyimage/")
def classifyimage(image64base : Data):
  print("in classify")
  if(image64base.file == ""): 
    return ["False"]
  
  with open("imageToSave.png", "wb") as fh:
    fh.write(base64.b64decode(image64base.file))
  img = Image.open("imageToSave.png")

  animal_name = get_animal_name(img)
  animal_name = animal_name.replace(" ", "_")
  animal_name = animal_name.lower()
  print(animal_name)
  names = animal_name.split(",_")
  t = Calssify()
  return t.classify(names[0])

In [ ]:
from nltk.corpus import wordnet as wn

pet = wn.synset('domestic_animal.n.01')
#pet = wn.synset('animal.n.01')
entity = wn.synset('entity.n.01')
bird = wn.synset('bird.n.01')

class Animal:
    PET  = 1
    BIRD = 2
    NONE = 3

class Calssify:
    def __init__(self):
        self.found = Animal.NONE
         
    def _DFSUtil(self,node):
        if(node == entity):
            self.found = Animal.NONE
            return
        if(node == bird):
            self.found = Animal.BIRD
            return
        if(node == pet):
            self.found = Animal.PET
            return               
        for neighbour in node.hypernyms():
            if node != entity:
                self._DFSUtil(neighbour)  

    def _DFS(self,node):
        self._DFSUtil(node)
        return self._printFound()
        
    def _printFound(self):
        if self.found == Animal.PET:
            print("IT IS A Pet")
            return ["True"]
        if self.found == Animal.BIRD:
            print("IT IS A BIRD")
            return ["True"]
        if self.found == Animal.NONE:
            print("NOT Domestic Animal")
            return ["False"]
    
    def classify(self,name):
        self.found = Animal.NONE
        if wn.synsets(name):
            print(name)
            name = name+".n.01"
            node = wn.synset(name)
            return self._DFS(node)
        else:
            print("NOT Animal")
            return ["False"]


In [ ]:
from torchvision import models
import torch

alexnet = models.alexnet(pretrained=True)
print(alexnet)

from torchvision import transforms
transform = transforms.Compose([            #[1]
  #transforms.ToPILImage(),
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 #transforms.RandomHorizontalFlip(), 
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
def get_animal_name(img):
  img_t = transform(img)
  batch_t = torch.unsqueeze(img_t, 0)
  out = alexnet(batch_t)
  print(out.shape)
  
  _, index = torch.max(out, 1)
  #percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
  #print(classes[index[0]], percentage[index[0]].item())
  print(classes[index[0]]) 
  animal_name = classes[index[0]] # we need to convert all letters to lower case and replace spaces with underscore
  return animal_name

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://0190093ae50f.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [61]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     156.208.228.22:0 - "POST /classifyimage HTTP/1.1" 307 Temporary Redirect
in classify
torch.Size([1, 1000])
hen
hen
hen
IT IS A BIRD
INFO:     156.208.228.22:0 - "POST /classifyimage/ HTTP/1.1" 200 OK


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


INFO:     156.208.228.22:0 - "POST /classifyimage HTTP/1.1" 307 Temporary Redirect
in classify
torch.Size([1, 1000])
lion, king of beasts, Panthera leo
lion,_king_of_beasts,_panthera_leo
lion
NOT Domestic Animal
INFO:     156.208.228.22:0 - "POST /classifyimage/ HTTP/1.1" 200 OK
INFO:     156.208.228.22:0 - "POST /classifyimage HTTP/1.1" 307 Temporary Redirect
in classify
INFO:     156.208.228.22:0 - "POST /classifyimage/ HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/protocols/http/h11_impl.py", line 394, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/fastapi/applications.py", line 208, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 159, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.7/dist-pa

In [ ]:
import json
x =  '{ "name":"John", "age":30, "city":"New York"}'
x = '{"base46":"content in the file"}'
y = json.loads(x)
print(y["base46"]) 

content in the file
